# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import holoviews
import geoviews
import cartopy

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cockburn town,21.4612,-71.1419,81.99,72,4,18.25,TC,1698497830
1,1,sinop,41.7711,34.8709,71.29,46,100,1.90,TR,1698497831
2,2,hobyo,5.3505,48.5268,83.07,69,100,12.86,SO,1698497832
3,3,lebu,-37.6167,-73.6500,53.55,95,100,24.87,CL,1698497832
4,4,olonkinbyen,70.9221,-8.7187,30.51,79,87,5.86,SJ,1698497833


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng","Lat",
    geo = True,
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    scale = 1,
    alpha = 0.5,
    title ="OSM"
)

# Display the map
humidity_map

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
ideal_weather = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
ideal_weather = city_data_df.loc[(city_data_df["Cloudiness"] ==0)]

# Drop any rows with null values
ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,blackmans bay,-43.0167,147.3167,50.74,68,0,1.01,AU,1698497835
19,19,petropavlovsk-kamchatsky,53.0452,158.6483,37.45,59,0,6.71,RU,1698497846
21,21,luderitz,-26.6481,15.1594,73.67,42,0,38.83,NaN,1698497848
35,35,isafjordur,66.0755,-23.1240,35.64,80,0,0.00,IS,1698497865
36,36,bethel,41.3712,-73.4140,66.65,87,0,6.49,US,1698497866
...,...,...,...,...,...,...,...,...,...,...
532,532,karaj,35.8355,51.0103,73.63,18,0,11.50,IR,1698498520
534,534,urumqi,43.8010,87.6005,47.35,57,0,2.24,CN,1698498525
541,541,copiapo,-27.3667,-70.3333,65.41,44,0,1.66,CL,1698498534
551,551,katsuren-haebaru,26.3369,127.8719,74.17,70,0,3.44,JP,1698498549


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,cockburn town,TC,21.4612,-71.1419,72,
1,sinop,TR,41.7711,34.8709,46,
2,hobyo,SO,5.3505,48.5268,69,
3,lebu,CL,-37.6167,-73.6500,95,
4,olonkinbyen,SJ,70.9221,-8.7187,79,
...,...,...,...,...,...,...
563,shakawe,BW,-18.3667,21.8500,12,
564,abashiri,JP,44.0213,144.2697,92,
565,shache,CN,38.4167,77.2406,24,
566,mount isa,AU,-20.7333,139.5000,10,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cockburn town - nearest hotel: The Salt Raker Inn
sinop - nearest hotel: No hotel found
hobyo - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
ellensburg - nearest hotel: Hotel Windrow
blackmans bay - nearest hotel: Villa Howden
yellowknife - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
harstad - nearest hotel: Scandic Harstad
vila velha - nearest hotel: Hotel Prainha
leninskoye - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
carnarvon - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
gadzhiyevo - nearest hotel: Оазис
toliara - nearest hotel: Ambary
borgarnes - nearest hotel: Borgarnes
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
iqaluit - nearest hotel: Frobisher Inn
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
sitka - neare

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng","Lat",  
                                      tiles="OSM", 
                                      geo=True, 
                                      color= "City",
                                      scale=1,
                                      alpha=0.5, 
                                      frame_height=500,
                                      frame_width=700, 
                                      size="Humidity", 
                                      hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_map